What can be used? <br>
PCA, imbalanced dataset, columns can be renamed into higher letter inthe beginning

In [3]:
!pip install bokeh

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from sklearn.ensemble import GradientBoostingClassifier as GB
#from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [11]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
# bokeh packages
from bokeh.io import output_file,show,output_notebook,push_notebook
from bokeh.plotting import *
from bokeh.models import ColumnDataSource,HoverTool,CategoricalColorMapper
from bokeh.layouts import row,column,gridplot,widgetbox
from bokeh.models.widgets import Tabs,Panel
from IPython.html.widgets import interact
output_notebook()

heart-failure-clinical-data



/opt/conda/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning:

The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.



Loading BokehJS ...

In [2]:
clinical_data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [4]:
#The explanation of the columns' meaning
#https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5/tables/1
clinical_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [5]:
print(clinical_data.shape)
clinical_data.isnull().sum()

(299, 13)


age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64

In [6]:
cont_features = [i for i in clinical_data.columns if len(clinical_data[i].unique())>2]
discrete_features = [i for i in clinical_data.drop('DEATH_EVENT',axis=1).columns\
                     if len(clinical_data[i].unique())<=2]

In [15]:
cont_features

['age',
 'creatinine_phosphokinase',
 'ejection_fraction',
 'platelets',
 'serum_creatinine',
 'serum_sodium',
 'time']

## Data Visualization with Bokeh

In [17]:
source = ColumnDataSource(clinical_data)

x,y = 'age','creatinine_phosphokinase'
plot = figure(x_axis_label = x,y_axis_label = y)
plot.circle(x=x,y=y,source = source)   #Can be added hue of death
show(plot)

In [18]:
# Selection appearance
plot = figure(tools="box_select,lasso_select")
plot.circle(x= x,y = y,source=source,color = "black",
            selection_color = "orange",
            nonselection_fill_alpha = 0.2,
           nonselection_fill_color = "blue")
show(plot)

In [31]:
data_died = clinical_data[clinical_data['DEATH_EVENT']==1]
data_survived = clinical_data[clinical_data['DEATH_EVENT']==0]

In [34]:
for i in cont_features[1:]:
    curdoc().theme = 'dark_minimal'

    # create and define the figure object
    x,y = 'age',i

    fig = figure(title=f'Age vs {y}', 
                 x_axis_label=x, 
                 y_axis_label=y,          
                 plot_width=750, plot_height=500,
                 tools= 'hover',# set this to None if you do not want data to be displayed on hover
                 toolbar_location="above", 
                 toolbar_sticky=False)

    fig.circle(x=x,y=y, 
             size=12, alpha=0.5, 
             color="#F78888", 
             legend_label='Not Survived', 
             source=data_died),
    fig.circle(x=x,y=y, 
             size=9, alpha=0.99, 
             color="green", 
             legend_label='Survived', 
             source=data_survived),

    fig.title.text_font_size = '20pt'
    fig.title.text_font_style = 'bold'
    fig.title.text_font = 'Serif'
    fig.xaxis.axis_label_text_font_size = "16pt"
    fig.yaxis.axis_label_text_font_size = "16pt"
    fig.legend.location = 'top_left'
    fig.legend.background_fill_color = "skyblue"

    show(fig)

By the last plot we can conclude that the longer follow-up period can lower the death event probability. 

In [35]:
len(cont_features)

7

In [53]:
curdoc().theme = 'dark_minimal'
titles = []
# create and define the figure object
x,y = 'age',cont_features[1]
title=f'Age vs {y}'
titles.append(title)
p1 = figure(title=title, 
             x_axis_label=x, 
             y_axis_label=y,          
             plot_width=750, plot_height=500,
             tools= 'hover',# set this to None if you do not want data to be displayed on hover
             toolbar_location="above", 
             toolbar_sticky=False)

p1.circle(x=x,y=y, 
         size=12, alpha=0.5, 
         color="#F78888", 
         legend_label='Not Survived', 
         source=data_died),
p1.circle(x=x,y=y, 
         size=9, alpha=0.99, 
         color="green", 
         legend_label='Survived', 
         source=data_survived),
#show(p1)

curdoc().theme = 'dark_minimal'

# create and define the figure object
x,y = 'age',cont_features[2]
title=f'Age vs {y}'
titles.append(title)
p2 = figure(title=title, 
             x_axis_label=x, 
             y_axis_label=y,          
             plot_width=750, plot_height=500,
             tools= 'hover',# set this to None if you do not want data to be displayed on hover
             toolbar_location="above", 
             toolbar_sticky=False)

p2.circle(x=x,y=y, 
         size=12, alpha=0.5, 
         color="#F78888", 
         legend_label='Not Survived', 
         source=data_died),
p2.circle(x=x,y=y, 
         size=9, alpha=0.99, 
         color="green", 
         legend_label='Survived', 
         source=data_survived),

x,y = 'age',cont_features[3]
title=f'Age vs {y}'
titles.append(title)
p3 = figure(title=title, 
             x_axis_label=x, 
             y_axis_label=y,          
             plot_width=750, plot_height=500,
             tools= 'hover',# set this to None if you do not want data to be displayed on hover
             toolbar_location="above", 
             toolbar_sticky=False)

p3.circle(x=x,y=y, 
         size=12, alpha=0.5, 
         color="#F78888", 
         legend_label='Not Survived', 
         source=data_died),
p3.circle(x=x,y=y, 
         size=9, alpha=0.99, 
         color="green", 
         legend_label='Survived', 
         source=data_survived),


x,y = 'age',cont_features[4]
title=f'Age vs {y}'
titles.append(title)
p4 = figure(title=title, 
             x_axis_label=x, 
             y_axis_label=y,          
             plot_width=750, plot_height=500,
             tools= 'hover',# set this to None if you do not want data to be displayed on hover
             toolbar_location="above", 
             toolbar_sticky=False)

p4.circle(x=x,y=y, 
         size=12, alpha=0.5, 
         color="#F78888", 
         legend_label='Not Survived', 
         source=data_died),
p4.circle(x=x,y=y, 
         size=9, alpha=0.99, 
         color="green", 
         legend_label='Survived', 
         source=data_survived),

x,y = 'age',cont_features[5]
title=f'Age vs {y}'
titles.append(title)
p5 = figure(title=title, 
             x_axis_label=x, 
             y_axis_label=y,          
             plot_width=750, plot_height=500,
             tools= 'hover',# set this to None if you do not want data to be displayed on hover
             toolbar_location="above", 
             toolbar_sticky=False)

p5.circle(x=x,y=y, 
         size=12, alpha=0.5, 
         color="#F78888", 
         legend_label='Not Survived', 
         source=data_died),
p5.circle(x=x,y=y, 
         size=9, alpha=0.99, 
         color="green", 
         legend_label='Survived', 
         source=data_survived),

x,y = 'age',cont_features[6]
title=f'Age vs {y}'
titles.append(title)
p6 = figure(title=title,
             x_axis_label=x, 
             y_axis_label=y,          
             plot_width=750, plot_height=500,
             tools= 'hover',# set this to None if you do not want data to be displayed on hover
             toolbar_location="above", 
             toolbar_sticky=False)

p6.circle(x=x,y=y, 
         size=12, alpha=0.5, 
         color="#F78888", 
         legend_label='Not Survived', 
         source=data_died),
p6.circle(x=x,y=y, 
         size=9, alpha=0.99, 
         color="green", 
         legend_label='Survived', 
         source=data_survived),

(GlyphRenderer(id='18203', ...),)

In [56]:
#Tabbed layout
#I use p1 and p2 that are created at above
tab1 = Panel(child = p1,title=titles[0])
tab2 = Panel(child = p2,title=titles[1])
tab3 = Panel(child = p3,title=titles[2])
tab4 = Panel(child = p4,title=titles[3])
tab5 = Panel(child = p5,title=titles[4])
tab6 = Panel(child = p6,title=titles[5])
tabs = Tabs(tabs = [tab1,tab2,tab3,tab4,tab5,tab6])
show(tabs)

In [8]:
print(clinical_data['DEATH_EVENT'].value_counts())
print(clinical_data['time'].value_counts())

0    203
1     96
Name: DEATH_EVENT, dtype: int64
250    7
187    7
10     6
186    6
107    6
      ..
97     1
96     1
86     1
77     1
285    1
Name: time, Length: 148, dtype: int64


In [9]:
clinical_data[['time','DEATH_EVENT']].corr()

,time,DEATH_EVENT
time,1.000000,-0.526964
DEATH_EVENT,-0.526964,1.000000


High correlation between time and Death

In [ ]:
sns.heatmap(clinical_data.corr(),annot=False)

In [ ]:
sns.catplot(x='DEATH_EVENT', y="age", kind="box", data=clinical_data)

In [ ]:
print(clinical_data['age'].min())
print(clinical_data['age'].max())

## Data Modelling

In [ ]:
X = clinical_data.iloc[:,:-1]
#X.drop('time',axis=1,inplace=True)
y = clinical_data.iloc[:,-1]
seed = 23
scoring = 'f1'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=seed)

models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [ ]:
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed,shuffle = True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    #msg = %s: %f (%f) % (name, cv_results.mean(), cv_results.std())
    print('{} {} {}'.format(name, cv_results.mean(), cv_results.std()))

In [ ]:
model = LinearDiscriminantAnalysis()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print(f1_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
kfold = KFold(n_splits=5, shuffle = True)
for i in range(10,200,10):

    clf = GB(n_estimators=i,random_state=seed)
    start_time = datetime.datetime.now()
    mean_score = cross_val_score(clf, X, y, scoring="roc_auc", cv = kfold).mean()
    #print ('Time elapsed:', datetime.datetime.now() - start_time)
    #print(mean_score)
    plt.scatter(i,mean_score)

In [ ]:
model = GB(n_estimators=50)#,learning_rate = )
model.fit(X_train,y_train)
model.score(X_test,y_test)